# Week4 Assignment Multiclass SVM 구현 Tobigs15th 이성범

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X = iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [ ]:
# 기존 코드가 전체 데이터를 바탕으로 스케일링이 되어있어서
# Train과 Test로 데이터를 나눈 후 Train 데이터를 기준으로 스케일링 작업을 했습니다.
# 또한 분류 데이터는 기본적으로 균등하게 나누어야 하기 때문에 stratify 옵션을 적용해야한다.
# 그러나 stratify 옵션을 적용하면 예측 확률이 1이 나와서 stratify 옵션을 적용하지 않았습니다.

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# OneVsRest

decision_function는 각 클래스에 대한 결정 함수를 나타내는 함수이다. SVC의 경우 0을 기준으로 0보다 크다며 해당 클래스로 예측을 한다. 즉 값이 높을 수록 해당 클래스가 될 확률이 높은 것이다.

OneVsRest는 각 클래스를 예측하는 모델 별로 이러한 결정 함수가 가장 큰 값을 선택하는 방식이다. 따라서 OneVsRest같은 경우 클래스의 개수 만큼의 모델이 필요하다.


#### sklearn의 SVC 공식 문서에서 제공하는 decision_function에 대한 설명

Evaluates the decision function for the samples in X.

#### OneVsRest에 대한 설명
1. https://wikidocs.net/4291
2. https://machinelearningmastery.com/one-vs-rest-and-one-vs-one-for-multi-class-classification/

In [ ]:
## Case 1 : One vs Rest
from sklearn.metrics import accuracy_score

def OneVsRest(X_train, X_test, y_train, y_test):
    # 클래스의 원핫 인코딩을 시행
    y_train = pd.get_dummies(y_train)
    
    # 클래스의 순서가 저장된 리스트
    class_list = y_train.columns.tolist()
    
    # 예측 값이 저장될 리스트
    y_pred = []
    # 클래스 만큼의 모델을 만들어줌
    for i in range(len(class_list)):
        # 모델 학습
        model = SVC(kernel ='rbf', C = 5, gamma = 5)
        
        # 해당 클래스를 학습한 모델
        model.fit(X_train, y_train.iloc[:,i])
        
        # 해당 클래스의 결정 함수를 구함
        decision_function = model.decision_function(X_test).reshape(-1,1)
        
        # 해당 클래스의 결정 함수을 저장
        y_pred.append(decision_function)
    
    # 모델별 결정 함수 를 합침
    y_pred = np.concatenate(y_pred, axis = 1)
    
    # 각 모델별 결정함수가 가장 큰 값의 인덱스를 반횐
    # 여기서의 인덱스가 곧 class_list의 인덱스가 됨
    # 동일한 값이 생길 경우 자동적으로 제일 먼저 등장한 값을 뽑아준다.
    y_pred = np.argmax(y_pred, axis=1)
    
    # 인덱스에 맞춰서 각 값을 실제 클래스 명으로 변환
    y_pred = [ class_list[i] for i in y_pred ]
    
    # accuracy_score 함수를 활용하기 위한 배열 변환
    y_pred = np.array(y_pred)
    
    # 에측값 반환
    return accuracy_score(y_pred, y_test), y_pred

scroe, y_pred = OneVsRest(X_train, X_test, y_train, y_test)
print(f"Class: {y_pred}\nAccuracy: {scroe}")

Class: ['versicolor' 'versicolor' 'versicolor' 'virginica' 'virginica'
 'virginica' 'setosa' 'virginica' 'setosa' 'versicolor' 'virginica'
 'setosa' 'setosa' 'virginica' 'versicolor' 'versicolor' 'setosa'
 'versicolor' 'virginica' 'virginica' 'setosa' 'virginica' 'versicolor'
 'versicolor' 'virginica' 'setosa' 'setosa' 'virginica' 'virginica'
 'versicolor']
Accuracy: 0.8666666666666667


In [ ]:
svc = SVC(kernel ='rbf', C = 5, gamma = 5)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
score = accuracy_score(y_pred, y_test)
print(f"Class: {y_pred}\nAccuracy: {scroe}")

Class: ['versicolor' 'versicolor' 'versicolor' 'virginica' 'virginica'
 'virginica' 'setosa' 'virginica' 'setosa' 'versicolor' 'virginica'
 'setosa' 'setosa' 'virginica' 'versicolor' 'versicolor' 'setosa'
 'versicolor' 'virginica' 'virginica' 'setosa' 'virginica' 'versicolor'
 'versicolor' 'virginica' 'setosa' 'setosa' 'virginica' 'virginica'
 'versicolor']
Accuracy: 0.8666666666666667


기존 모델과 동일한 결과를 반환한다는 것을 알 수 있음

# OneVsOne

다중 클래스 분류 데이터 세트를 이진 분류 데이터 세트들로 분활하여 투표를 통해서 가장 다득표를 받은 클래스를 선택하는 방식이다. 따라서 모델은 이진 분류 데이터 세트 만큼이 필요하며 수식적으로는 $ _{m}C_{2} $ 개의 모델이 필요하며 여기서 m은 클래스의 개수를 의미한다. 또한 각 클래스 별로 득표를 받을 수 있는 최대 투표수는 $ _{m}C_{2} - 1 $ 개 이다. 

#### OneVsOne에 대한 설명
1. https://machinelearningmastery.com/one-vs-rest-and-one-vs-one-for-multi-class-classification/

In [ ]:
# Case 2 : One vs One
from itertools import combinations

def OneVsOne(X_train, X_test, y_train, y_test):
    
    # 클래스가 저장된 리스트
    class_list = np.unique(y_train).tolist()
    
    # 이진 분류된 클래스가 저장될 리스트 
    binary_class_list = list(map(list, combinations(class_list, 2)))
    
    # 투표가 저장될 데이터 프레임
    # 행은 테스트 데이터의 개수 만큼, 열은 클래스의 개수 만큼 생성
    votes_df = pd.DataFrame(np.zeros((len(X_test), len(class_list))), columns = class_list)
    
    # 이진 분류 데이터 셋으로 나눈다.
    # 이진 븐류된 데이터 만큼의 반복을 해야함
    for binary_class in binary_class_list:
        class1, class2 = binary_class
        X_train_binary = X_train[(y_train == class1) | (y_train == class2)]
        y_train_binary = y_train[(y_train == class1) | (y_train == class2)]
        
        model = SVC(kernel ='rbf', C = 5, gamma = 5)
        model.fit(X_train_binary, y_train_binary)
        class_pred_list = model.predict(X_test)
        
        # 데이터 별로 예측한 값에 대해서 클래스별로 투표를 함
        for idx, class_pred in enumerate(class_pred_list):
            votes_df.loc[idx, [class_pred]] += 1
    
    # test 데이터에 대한 클래스별 투표 수를 배열화함
    y_pred = votes_df.values
    
    # test 데이터에 대한 클래스별 투표 수가 가장 큰 값의 인덱스를 반횐
    # 여기서의 인덱스가 곧 class_list의 인덱스가 되고
    # 그 인덱스가 곧 해당 클래스가 됨
    # 동일한 값이 생길 경우 자동적으로 제일 먼저 등장한 값을 뽑아준다.
    y_pred = np.argmax(y_pred, axis=1)
    
    # 인덱스에 맞춰서 각 값을 실제 클래스 명으로 변환
    y_pred = [ class_list[i] for i in y_pred ]
    
    # accuracy_score 함수를 활용하기 위한 배열 변환
    y_pred = np.array(y_pred)

    return accuracy_score(y_pred, y_test), y_pred, votes_df

scroe, y_pred, votes_df = OneVsOne(X_train, X_test, y_train, y_test)
print(f"Class: {y_pred}\nAccuracy: {scroe}")
print('투표 결과')
votes_df

Class: ['versicolor' 'versicolor' 'versicolor' 'virginica' 'virginica'
 'virginica' 'setosa' 'virginica' 'setosa' 'versicolor' 'virginica'
 'setosa' 'setosa' 'virginica' 'versicolor' 'versicolor' 'setosa'
 'versicolor' 'virginica' 'virginica' 'setosa' 'virginica' 'versicolor'
 'versicolor' 'virginica' 'setosa' 'setosa' 'virginica' 'virginica'
 'versicolor']
Accuracy: 0.8666666666666667
투표 결과


,setosa,versicolor,virginica
0,0.0,2.0,1.0
1,0.0,2.0,1.0
2,0.0,2.0,1.0
3,0.0,1.0,2.0
4,0.0,1.0,2.0
5,0.0,1.0,2.0
6,2.0,0.0,1.0
7,0.0,1.0,2.0
8,2.0,0.0,1.0
9,0.0,2.0,1.0


In [ ]:
svc = SVC(kernel ='rbf', C = 5, gamma = 5)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
score = accuracy_score(y_pred, y_test)
print(f"Class: {y_pred}\nAccuracy: {scroe}")

Class: ['versicolor' 'versicolor' 'versicolor' 'virginica' 'virginica'
 'virginica' 'setosa' 'virginica' 'setosa' 'versicolor' 'virginica'
 'setosa' 'setosa' 'virginica' 'versicolor' 'versicolor' 'setosa'
 'versicolor' 'virginica' 'virginica' 'setosa' 'virginica' 'versicolor'
 'versicolor' 'virginica' 'setosa' 'setosa' 'virginica' 'virginica'
 'versicolor']
Accuracy: 0.8666666666666667


기존 모델과 동일한 결과를 반환한다는 것을 알 수 있음